<a href="https://colab.research.google.com/github/diego-ciciani01/ADM---HV3/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
url = 'https://www.findamasters.com/masters-degrees/msc-degrees/'

result = requests.get(url)
result.text

In [ ]:
soup = BeautifulSoup(result.text)
soup.prettify() # this command will transform the result into a file that we can browse it
# use beautifulsoup tutorial https://beautiful-soup-4.readthedocs.io/en/latest/

In [ ]:
list_links = soup.find_all('a', href=True) # will produce a list of all the links that extracts in the web page

In [ ]:
# we can iterate on it and find what we need filtering all of them
# the links we need start all with the same prefix
prefix = '/masters-degrees/course/'
results = [] 

# we have different links that get to the same page (and we'll need to exclude them) 
# so we'll create a list of things to exclude
exclude = ['\nMore details \n', '\nRead more \n', '\xa0Video(s)', '\xa0Student Profile(s)']

for item in list_links: #using the href to extract the links 
    if item['href'][:len(prefix)] == prefix and not item.text in exclude:
        results.append((item['href'], item.text))

results


In [ ]:
lst_links = soup.find_all('a', {'class': 'courseLink'}) #add attribute we find this attribute under the href category 

In [ ]:
def extract_masters(this_url):
    result_url=requests.get(this_url)
    result_soup=BeautifulSoup(result_url.text)
    result_links=result_soup.find_all('a',{'class':'courseLink'})
    result_list=[]
    for item in result_links:
        result_list.append((item['href'],item.text))
    return result_list

In [ ]:
test_lst = extract_masters('https://www.findamasters.com/masters-degrees/msc-degrees/?PG=2')

In [ ]:
msc_page_url = 'https://www.findamasters.com' + test_lst[0][0]

In [ ]:
page_url = requests.get(msc_page_url)
page_soup = BeautifulSoup(page_url.text)

In [ ]:
page_links = page_soup.find_all('a', {'class': 'course-header__institution'})
page_universityName = page_links[0].contents[0]

In [ ]:
page_universityName

In [ ]:
def extract_msc_page(msc_page_url):
  contents = {}

  page_url = requests.get(msc_page_url)
  page_soup = BeautifulSoup(page_url.text)

  page_links = page_soup.find_all('a', {'class': 'course-header__institution'})
  contents['universityName'] = page_links[0].contents[0]

  page_links = page_soup.find_all('a', {'class': 'course-header__department'})
  contents['facultyName'] = page_links[0].contents[0]

  FullTime_links= page_soup.find_all('a', {'class':'concealLink' })
  FullTime = False
  for item in FullTime_links:
    if item['href']== "/masters-degrees/full-time/":
      FullTime = True
      break
  contents['fullTime'] = FullTime

  return contents

In [ ]:
pages_results = []
for item in test_lst:
  print('retrieving ' + item[0])
  result_msc_page = extract_msc_page('https://www.findamasters.com' + item[0])
  result_msc_page['name'] = item[1]
  pages_results.append(result_msc_page)

In [ ]:
pages_results

In [ ]:
import pandas as pd
pd.DataFrame.from_records(pages_results)